In [1]:
import json
import os
import pandas as pd
import ahpy
import process_engine.stats as pes
import process_engine.location_modeller as lmd
import shapely as sh

In [5]:
with open(os.path.join(os.getcwd(), 'raw_data', 'sample6.json'), 'r') as file:
    data = json.load(file)


cleaned = {}

for key, value in data.items():
    for subkey, subvalue in value.items():
        importancy = subvalue['importancy']
        scale = subvalue['scale']
        if (importancy == 'A'):
            cleaned[(key, subkey)] = scale
        else:
            cleaned[(key, subkey)] = pes.reciprocical_value(int(scale))


shelter = ahpy.Compare(name='shelter', comparisons=cleaned,
                       precision=3, random_index='saaty')

report = shelter.report()

In [6]:
with open('ahp_report.json', 'w') as rep:
    rep.write(json.dumps(report))

### Sensitivity Analysis


The sensitivity analysis will be based on the adjusting AHP derived wieghts


In [3]:
from process_engine.oat import OAT

In [4]:
weight_dict = report['target_weights']
oat_model = OAT(weight_dict, 5)

In [5]:
# Derive Sensitivity Matrix
sm = oat_model.solve(100, -100, 25)

[-0.75, -0.5, -0.25, 0.0, 0.25, 0.5, 0.75]


In [11]:
sm[0][6]

array([0.3815    , 0.15818414, 0.13682928, 0.12417455, 0.08146483,
       0.03954604, 0.03084591, 0.02610038, 0.02135486])

### Geo Spatial Analysis


In [7]:
import geopandas as gpd
from process_engine.vector import addCellsToGDF
import shapely as sh

In [8]:
tas = gpd.read_file('raw_data/vector_database.gpkg',
                    layer='traditional_authorities')
tas_wgs84 = tas.to_crs('EPSG:4326')
tas_cells = addCellsToGDF(tas_wgs84, 8)
tas_cells = tas_cells.explode('cell_features', ignore_index=True)
tas_cells['geometry'] = tas_cells['cell_features']

In [9]:
tas_cells

,_uid_,id,ta_name,dist_name,geometry,cells,cell_features
0,1,299,Chikwawa Boma,Chikwawa,"MULTIPOLYGON (((34.78501 -16.00913, 34.77966 -...","[8897ad4991fffff, 8897ad498bfffff, 8897ad4995f...",MULTIPOLYGON (((34.78500803278594 -16.00913181...
1,1,299,Chikwawa Boma,Chikwawa,"MULTIPOLYGON (((34.80255 -16.01547, 34.79721 -...","[8897ad4991fffff, 8897ad498bfffff, 8897ad4995f...",MULTIPOLYGON (((34.80255431125956 -16.01546516...
2,1,299,Chikwawa Boma,Chikwawa,"MULTIPOLYGON (((34.78501 -16.00913, 34.78844 -...","[8897ad4991fffff, 8897ad498bfffff, 8897ad4995f...",MULTIPOLYGON (((34.78500803278594 -16.00913181...
3,1,299,Chikwawa Boma,Chikwawa,"MULTIPOLYGON (((34.78419 -16.03677, 34.78953 -...","[8897ad4991fffff, 8897ad498bfffff, 8897ad4995f...",MULTIPOLYGON (((34.78418718021861 -16.03677478...
4,1,299,Chikwawa Boma,Chikwawa,"MULTIPOLYGON (((34.80105 -16.00654, 34.7957 -1...","[8897ad4991fffff, 8897ad498bfffff, 8897ad4995f...",MULTIPOLYGON (((34.80104535120539 -16.00653905...
...,...,...,...,...,...,...,...
5598,15,310,TA Maseya,Chikwawa,"MULTIPOLYGON (((34.78021 -16.11077, 34.77679 -...","[8897ad4a6dfffff, 8897ad4b15fffff, 8897ad59b1f...",MULTIPOLYGON (((34.78021452603731 -16.11077170...
5599,15,310,TA Maseya,Chikwawa,"MULTIPOLYGON (((34.89482 -16.07387, 34.89674 -...","[8897ad4a6dfffff, 8897ad4b15fffff, 8897ad59b1f...",MULTIPOLYGON (((34.89482089703952 -16.07387375...
5600,15,310,TA Maseya,Chikwawa,"MULTIPOLYGON (((34.79887 -16.13984, 34.8023 -1...","[8897ad4a6dfffff, 8897ad4b15fffff, 8897ad59b1f...",MULTIPOLYGON (((34.798868209513586 -16.1398443...
5601,15,310,TA Maseya,Chikwawa,"MULTIPOLYGON (((34.80312 -16.11624, 34.79777 -...","[8897ad4a6dfffff, 8897ad4b15fffff, 8897ad59b1f...",MULTIPOLYGON (((34.80311646485109 -16.11623623...


In [10]:
tas_final = gpd.GeoDataFrame(
    tas_cells[['ta_name', 'dist_name', 'cell_features']], crs='4326', geometry='cell_features')
tas_final = tas_final.to_crs('EPSG:20936')
tas_final.to_file('raw_data/vector_database.gpkg',
                  driver='GPKG', layer='tas_hex', mode='w')

### Location Allocation Modelling


In [1]:
import pandas as pd
import process_engine.location_modeller as lmd
import shapely as sh

In [2]:
df = pd.read_csv("raw_data/shelter_victim_matrix_walking_V8.csv", dtype={
    'FROM_ID': str,
    'TO_ID': str,
    'DURATION_H': 'float64',
    'DIST_KM': 'float64',
    'DURATION_MIN': 'float64',
    'DURATION_SEC': 'float64'
})

In [3]:
solver = lmd.LSCP_Solver(od_matrix=df, dest_col="TO_ID",
                         origin_col="FROM_ID",  value_col="DURATION_H")

In [5]:
solver.solve(coverage=2, coverage_percentage=1)


Coverage Analysis (distance ≤ 2):
  Facilities covering each demand (avg): 31.09
  Min facilities covering any demand: 2
  Max facilities covering any demand: 113
  ✓ All demand points can be covered
required 55.0

SOLVING LSCP WITH OD MATRIX
Demand points: 55
Candidate facilities: 1111

Solving...
-1
No solution found. Status: Infeasible

Possible reasons:
1. Coverage distance too small - no facilities can cover enough demand
2. Coverage percentage too high - try lowering it
3. Check your distance matrix for errors


{'status': 'Infeasible'}

In [5]:
df_x, df_y = solver.export_results()

ValueError: Solve the problem first!

In [23]:
hex_lookup = pd.read_csv('raw_data/hex_lookup.csv')

In [24]:
df_y['demand_coord'] = df_y['demand_index'].apply(lambda x: hex_lookup.set_index(
    'place_id').loc[x, 'center_coord'])
df_y['facility_coord'] = df_y['nearest_facilities'].apply(lambda x: hex_lookup.set_index(
    'place_id').loc[x, 'center_coord'])

In [25]:
# df_y.apply(lambda x: sh.linestrings([sh.from_wkt(x['demand_coord']), sh.from_wkt(x['facility_coord'])]), axis=1)
def getLineString(x):
    start = sh.get_coordinates(sh.from_wkt(x['demand_coord']))[0]
    end = sh.get_coordinates(sh.from_wkt(x['facility_coord']))[0]
    listString = sh.LineString([start, end])
    return sh.to_wkt(listString)


df_y['geom'] = df_y.apply(lambda x: getLineString(x), axis=1)

In [26]:
df_y.to_csv('lscp_solution_assignments.csv')

In [43]:
solver.origin_dict

{'chikwawa_boma_C1_10': 0,
 'ngabu_urban_C1_2984': 1,
 'ngabu_urban_C1_2987': 2,
 'ngabu_urban_C1_2988': 3,
 'sta_masache_C1_3100': 4,
 'sta_ndakwela_C1_3194': 5,
 'sta_ndakwela_C1_3259': 6,
 'ta_chapananga_C1_3423': 7,
 'ta_chapananga_C1_3469': 8,
 'ta_chapananga_C1_3688': 9,
 'ta_chapananga_C1_3899': 10,
 'ta_chapananga_C1_3917': 11,
 'ta_chapananga_C1_3940': 12,
 'ta_katunga_C1_4583': 13,
 'ta_katunga_C1_4646': 14,
 'ta_katunga_C1_4656': 15,
 'ta_katunga_C1_4661': 16,
 'ta_maseya_C1_5506': 17,
 'ta_maseya_C1_5571': 18,
 'ta_maseya_C1_5573': 19,
 'ta_ngabu_C1_1990': 20,
 'ta_ngabu_C1_2079': 21,
 'ta_ngabu_C1_2281': 22,
 'ta_ngabu_C1_2349': 23,
 'ta_ngabu_C1_2391': 24,
 'ta_ngabu_C1_2565': 25,
 'ta_ngabu_C1_2566': 26,
 'ta_ngowe_C1_2917': 27,
 'ta_ngowe_C1_2966': 28}

In [45]:
solver.dest_dict

{'chikwawa_boma_A1_4': 0,
 'lengwe_national_park_A1_102': 1,
 'lengwe_national_park_A1_164': 2,
 'lengwe_national_park_A1_193': 3,
 'lengwe_national_park_A1_222': 4,
 'lengwe_national_park_A1_226': 5,
 'lengwe_national_park_A1_246': 6,
 'lengwe_national_park_A1_274': 7,
 'lengwe_national_park_A1_287': 8,
 'lengwe_national_park_A1_292': 9,
 'lengwe_national_park_A1_351': 10,
 'lengwe_national_park_A1_452': 11,
 'lengwe_national_park_A1_468': 12,
 'lengwe_national_park_A1_472': 13,
 'lengwe_national_park_A1_505': 14,
 'lengwe_national_park_A1_513': 15,
 'lengwe_national_park_A1_587': 16,
 'lengwe_national_park_A1_624': 17,
 'lengwe_national_park_A1_625': 18,
 'lengwe_national_park_A1_682': 19,
 'lengwe_national_park_A1_730': 20,
 'lengwe_national_park_A1_751': 21,
 'lengwe_national_park_A1_77': 22,
 'lengwe_national_park_A1_89': 23,
 'majete_game_reserve_-_chikwawa_A1_1169': 24,
 'majete_game_reserve_-_chikwawa_A1_1446': 25,
 'majete_game_reserve_-_chikwawa_A1_1515': 26,
 'majete_game_re